# Web scraping script:

How is the data used in the analysis obtained? This script presents the web scraping part that consist in a technique used to extract data from websites and store it in a json file: I used OOP the create a simple class that can scrape Amazon reviews based on the ASIN (Amazon's product ID) given in input.

---

In this case I will scrape the reviews on a product which is a pair of speakers.

Reviews URL: 'https://www.amazon.co.uk/product-reviews/B075QVMBT9/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=1'

---
#### Steps:
- Get the URL of the page to be scrapped.
- Inspect the elements of the page and identify the tags required.
- Access the URL.
- Get the element from the required tags.

## Importing necessary dependencies

In [8]:
from requests_html import HTMLSession
import json
import time

### Class explanation:
- **what is asin?** For amazon, the ASIN (Amazon Standard Identification Number) is basically the product identifier.
- **User agents:** When scraping a website, you also need to set user-agents on every request as otherwise the website may block your requests because it knows you aren't a real user. This user agent will clearly identify your requests as coming from a web scraper, so the website can easily block you from scraping the site. User-agent found on by googling 'my user agent'.
- **URL:** For the self.url variable i replaced the asin with the self.asin variable to make this script available for all products (given the asin of it). Note also that in the last part of the URL I left the 'pageNumber=' make it easier to iterate over all the reviews pages, that have different URLs.
---

For the **pagination** class method we use the session object created before to get the new url with the new page (obtained by concatenating the 2 strings self.url and the page number). The if means the following: if we don't find any reviews on the URL, return False (so when using this method in a loop, if its false we can break out the loop), otherwise return the response. This is going ot be really useful when lopping through many pages without knowing how many pages there are.

---

The **parse** method pulls out all the needed info, puts everything into a dictionary and that append it to a list which will return a list of many dictionaries which basically are all the reviews with their titles, star ratings and content. When runnign the code I incurred in many different errors, so i decided to put a try/except to handle these problems.

---

Lastly, the **save** method allows me to save the results in a json file.

---

**Note:** For selecting the needed pieces of the 'div' tag i used the CSS selector syntax for simplicity (https://requests.readthedocs.io/projects/requests-html/en/latest/) thanks to the requests_html library. This allows me to use the .find() method in a much more cleaner way rather than putting the class pf the 'div' element which is usually long and more complex, so for example:
- a[target=value] where 'a' is the tag in which i am 'navigating' and in the brackets we have some class and its value that i want to select.

In [151]:
class Reviews:
    def __init__(self, asin) -> None:
        self.session = HTMLSession()        # session object that is used continuously whenever this class instance is called
        self.headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}
        self.asin = asin
        self.url = f'https://www.amazon.co.uk/product-reviews/{self.asin}/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber='


    def pagination(self, page):
        response = self.session.get(self.url + str(page), headers=self.headers)     # getting the new page URLs
        # check if there are reviews in the new url o not, using CSS selectors
        if not response.html.find('div[data-hook=review]'):
            return False
        else:
            return response.html.find('div[data-hook=review]')
    

    def parse(self, reviews):
        total = []
        # looping on reviews to parse the data i will extract and put in a .json file later on
        for review in reviews:
            # .find() outputs a list, so with first=True i say that i want the first element of that list
            # .text to convert into text (string)
            try:
                title = review.find('a[data-hook=review-title] span', first=True).text
                rating = review.find('i[data-hook=review-star-rating] span', first=True).text   
                # span means i'm selecting the value of the element i[data-hook=review-star-rating] -> basically here i'm getting the number of stars of the review
                content = review.find('span[data-hook=review-body] span', first=True).text.replace('/n', '').strip()
                # .replace('/n') and .strip() to eliminate backslashes and clean the text
            except:
                continue        # for sure will end up with some missing pages or data to be cleaned

            data = {'title': title,
                    'rating': rating,
                    'body': content[:500]}      # some reviews are really long so maybe it it better to take just part of them
            total.append(data)
        return total
    

    def save(self, results):
        # opening a new file named self.asin + '_reviews.json', in which i will write ('w') all the results
        # assigning the new file to a new variable called f
        with open(self.asin + '_reviews.json', 'w') as f:
            json.dump(results, f)


### Scraping time!

In [152]:
if __name__ == '__main__':
    amz = Reviews('B075QVMBT9')
    results = []
    for i in range(1,101):   # 101 => a url provides n reviews with 10 reviews on each page --> therefore i take the first 1000 reviews
        print('Getting page', i)
        time.sleep(0.3)      # to let the loop "breathe"
        reviews = amz.pagination(i)
        if reviews:          # if there is something into reviews (seeing the many errors i've gotten) -> append it to the results
            results.append(amz.parse(reviews))

Getting page 1
Getting page 2
Getting page 3
Getting page 4
Getting page 5
Getting page 6
Getting page 7
Getting page 8
Getting page 9
Getting page 10
Getting page 11
Getting page 12
Getting page 13
Getting page 14
Getting page 15
Getting page 16
Getting page 17
Getting page 18
Getting page 19
Getting page 20
Getting page 21
Getting page 22
Getting page 23
Getting page 24
Getting page 25
Getting page 26
Getting page 27
Getting page 28
Getting page 29
Getting page 30
Getting page 31
Getting page 32
Getting page 33
Getting page 34
Getting page 35
Getting page 36
Getting page 37
Getting page 38
Getting page 39
Getting page 40
Getting page 41
Getting page 42
Getting page 43
Getting page 44
Getting page 45
Getting page 46
Getting page 47
Getting page 48
Getting page 49
Getting page 50
Getting page 51
Getting page 52
Getting page 53
Getting page 54
Getting page 55
Getting page 56
Getting page 57
Getting page 58
Getting page 59
Getting page 60
Getting page 61
Getting page 62
Getting page 63
G

Now all the desired pages are scraped but there are for sure some pages that were blocked by Amazon.

In [157]:
len(results[73])

8

In [158]:
len(results[74])

0

In [155]:
for i in range(0,100):
    if len(results[i]) == 0:
        print(len(results[i]), i)

0 74
0 75
0 76
0 77
0 78
0 79
0 80
0 81
0 82
0 83
0 84
0 85
0 86
0 87
0 88
0 89
0 90
0 91
0 92
0 93
0 94
0 95
0 96
0 97
0 98
0 99


Looks like it ended up with a solid result! Nearly 730 reviews (728 to be precise)! Proceeding now to save the reviews in a json file which will be later converted to .csv and then Pandas DataFrame to make the analysis.

In [125]:
amz.save(results)